In [ ]:
from transformers import RobertaConfig, RobertaModel, pipeline, RobertaTokenizer
import pandas as pd
import torch

In [ ]:
train_set = pd.read_csv('./data/Corona_NLP_train_clean.csv')
test_set = pd.read_csv('./data/Corona_NLP_test_clean.csv')

In [ ]:
# Initializing a RoBERTa configuration
configuration = RobertaConfig()

# Initializing a model from the configuration
model = RobertaModel.from_pretrained("roberta-base")

# Accessing the model configuration
configuration = model.config

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
X_train = list(train_set['OriginalTweet'].apply(lambda x: "[CLS]" + x + "[SEP]").values)

In [ ]:
X_train = tokenizer(X_train)

In [ ]:
X_train.keys()

In [ ]:
len(X_train['input_ids'][0])

In [ ]:
len(X_train['attention_mask'][0])

In [ ]:
max_len = 0
lengths = list()
for seq in X_train['input_ids']:
    if len(seq) > max_len:
        max_len = len(seq)
    lengths.append(len(seq))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(lengths)

In [ ]:
max_len

In [ ]:
def make_padded_tensor(ids, limit):
    for i, seq in enumerate(ids['input_ids']):
        if len(seq) > limit:
            ids['input_ids'][i] = seq[:limit]
            ids['attention_mask'][i] = ids['attention_mask'][i][:limit]
        else:
            ids['input_ids'][i] = seq + [0] * (limit - len(seq))
            ids['attention_mask'][i] = ids['attention_mask'][i] + [0] * (limit - len(seq))
    ids['input_ids'] = torch.LongTensor(ids['input_ids'])
    ids['attention_mask'] = torch.LongTensor(ids['attention_mask'])
    return ids

In [ ]:
X_tensors = make_padded_tensor(X_train, 100)

In [ ]:
X_tensors['input_ids'].shape

In [ ]:
stuff = model(X_tensors['input_ids'], X_tensors['attention_mask'])

In [ ]:
stuff[0].shape